## Preparation

In [33]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import wrds
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import preprocessing, tree
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
pd.set_option('display.max_columns', None)
import statsmodels.api as sm
from statsmodels.api import add_constant
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
from CompanyTextSentimentAnalyzer import TextSentimentAnalyser


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\r3080\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\r3080\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
db = wrds.Connection(wrds_username = 'rotmanren') # Change to user's username

Loading library list...
Done


In [35]:
# Load all data
startYear = 2009
# Accruals need data from 2009 to obtain changes
endYear = 2022
dataquery1 = f"""
            SELECT a.DATADATE, a.cik, a.GVKEY, a.tic, a.fyear, a.ib, a.spi, a.at, a.dvc, a.act, a.che, a.lct, a.dlc, a.ni, a.revt, a.txp, a.dp, a.csho, a.ceq, a.ivao, a.lt, a.ajex, a.dltt, a.ivst, a.pstk, b.cshom, b.prccm
            FROM comp.funda as a
            JOIN comp_na_daily_all.secm as b
            ON a.tic = b.tic AND a.fyear = b.cyear
            WHERE consol = 'C'
            AND popsrc = 'D'
            AND indfmt = 'INDL'
            AND datafmt = 'STD' 
            AND fyear BETWEEN '{startYear}' AND '{endYear}'
            AND b.cmth = 6.0
        """
inforaw = db.raw_sql(dataquery1)
# Fill NA for selected variables 
cols = ['spi', 'dvc', 'che', 'lct', 'dlc', 'txp', 'dp', 'ceq', 'ivao', 'lt', 'dltt', 'ivst', 'pstk']
inforaw[cols] = inforaw[cols].fillna(0)
inforaw = inforaw.dropna()
inforaw = inforaw.reset_index()
# Adjust for stock split
inforaw['csho'] = inforaw['csho'] * inforaw['ajex']
inforaw['mrk_eq'] = inforaw['cshom'] * inforaw['prccm']
inforaw


,index,datadate,cik,gvkey,tic,fyear,ib,spi,at,dvc,act,che,lct,dlc,ni,revt,txp,dp,csho,ceq,ivao,lt,ajex,dltt,ivst,pstk,cshom,prccm,mrk_eq
0,0,2010-05-31,0000001750,001004,AIR,2009.0,44.628,-4.302,1501.042,0.000,863.429,79.370,325.550,100.833,44.628,1352.151,3.263,38.930,39.4840,746.906,2.143,754.692,1.0,336.191,0.000,0.0,38697000.0,16.05,6.210868e+08
1,1,2011-05-31,0000001750,001004,AIR,2010.0,73.139,-1.536,1703.727,2.983,913.985,57.433,416.010,114.075,69.826,1775.782,0.000,59.296,39.7810,835.845,2.443,868.438,1.0,329.802,0.000,0.0,39025000.0,16.74,6.532785e+08
2,2,2012-05-31,0000001750,001004,AIR,2011.0,67.723,-13.864,2195.653,12.081,1063.272,67.720,473.226,122.865,67.723,2074.498,0.000,80.333,40.2730,864.649,18.869,1329.631,1.0,669.489,0.000,0.0,39739000.0,27.09,1.076530e+09
3,3,2013-05-31,0000001750,001004,AIR,2012.0,55.000,-21.100,2136.900,11.900,1033.700,75.300,389.000,86.400,55.000,2167.100,0.000,108.600,39.3820,918.600,16.800,1217.400,1.0,622.200,0.000,0.0,40288000.0,13.48,5.430822e+08
4,4,2014-05-31,0000001750,001004,AIR,2013.0,72.900,0.000,2199.500,11.800,1116.900,89.200,402.100,69.700,72.900,2035.000,0.000,113.400,39.5600,999.500,5.200,1198.800,1.0,564.300,0.000,0.0,39733000.0,21.98,8.733313e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68880,127165,2019-12-31,0001597659,335466,HOFSQ,2019.0,-19.116,0.000,201.909,0.000,23.703,12.681,8.083,0.207,-19.116,41.813,0.000,10.657,25.6620,52.128,0.000,149.781,1.0,141.698,0.000,0.0,20162000.0,2.75,5.544550e+07
68881,127166,2021-12-31,0001718939,345764,IDAI,2021.0,-9.057,0.000,8.665,0.000,5.764,3.476,2.400,0.000,-9.057,3.678,0.000,0.574,4.0384,4.872,0.178,3.631,0.2,0.856,0.000,0.0,3589000.0,14.25,5.114325e+07
68882,127167,2022-12-31,0001718939,345764,IDAI,2022.0,-12.092,0.000,6.412,0.000,2.874,1.255,4.448,0.297,-12.092,5.385,0.021,0.760,4.8544,0.464,0.000,5.787,0.2,1.077,0.000,0.0,23561000.0,1.69,3.981809e+07
68883,127168,2021-12-31,0001326110,347007,IBRX,2021.0,-346.790,-12.856,468.910,0.000,334.347,317.116,368.995,302.247,-346.790,0.934,0.000,14.238,397.8300,-242.173,0.000,712.823,1.0,343.417,136.015,0.0,383906000.0,14.28,5.482178e+09


In [36]:
# Predictions from analyst in year 2019, 2022
startYear_an = 2019
endYear_an = 2019
dataquery_one = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest, a.fpi
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '1'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_one)
linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
df_analyst_raw_test_2019 = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_raw_test_2019 = df_analyst_raw_test_2019.dropna()
df_analyst_raw_test_2019 = df_analyst_raw_test_2019.reset_index()
df_analyst_raw_test_2019

startYear_an = 2022
endYear_an = 2022
dataquery_one = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest, a.fpi
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '1'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_one)
linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
df_analyst_raw_test_2022 = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_raw_test_2022 = df_analyst_raw_test_2022.dropna()
df_analyst_raw_test_2022 = df_analyst_raw_test_2022.reset_index()
df_analyst_raw_test_2022

,index,ticker,fpedats,statpers,actual,medest,fpi,gvkey,TICKER
0,0,001K,2022-12-31,2022-06-16,8.95,7.31,1,19516,001K
1,1,001M,2022-12-31,2022-06-16,-1.40,-1.58,1,19274,001M
2,2,003J,2022-12-31,2022-06-16,0.22,0.55,1,19851,003J
3,3,003K,2022-06-30,2022-06-16,1.31,1.34,1,20115,003K
4,4,006X,2022-12-31,2022-06-16,0.47,0.90,1,20644,006X
...,...,...,...,...,...,...,...,...,...
2828,3522,YYIN,2022-12-31,2022-06-16,2.54,1.44,1,16011,YYIN
2829,3524,ZEN,2022-12-31,2022-06-16,3.33,3.30,1,28272,ZEN
2830,3525,ZM,2023-01-31,2022-06-16,4.37,3.74,1,35005,ZM
2831,3527,ZS,2022-07-31,2022-06-16,0.69,0.65,1,33037,ZS


In [37]:
# startYear_an = 2010
# endYear_an = 2022
# dataquery_one = f"""
#             SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest, a.fpi
#             FROM ibes.statsum_epsus as a
#             WHERE 
#             fpi = '1'
#             AND measure = 'EPS'
#             AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
#             AND EXTRACT(MONTH FROM a.statpers) = 6
#         """
# df_inspeciton = db.raw_sql(dataquery_one)
# linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
# # df_inspeciton = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
# df_inspeciton = df_inspeciton.dropna()
# df_inspeciton = df_inspeciton.reset_index()
# df_inspeciton
# df_inspeciton = pd.merge(inforaw, df_inspeciton, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])
# df_inspeciton['EPS'] = (df_inspeciton['ib'] - df_inspeciton['spi']) / df_inspeciton['csho']


## ADV preparation

In [38]:
# A function taking care of all financial values calculation
def  calValues(db, company_list):
    print('Estimated waiting time 4mins, please wait')
    # Prepare dataset for nine models
    # HVZ one year
    df_HVZ_one = pd.DataFrame(columns= ['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'tic', 'cik', 'fyear','mrk_eq'])
    # HVZ two year
    df_HVZ_two = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_2'])
    # HVZ three year
    df_HVZ_three = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_3'])

    # EP one year
    df_EP_one = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_1'])
    # EP two year
    df_EP_two = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_2'])
    # EP three year
    df_EP_three = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_3'])

    # RI one year
    df_RI_one = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_1'])
    # RI two year
    df_RI_two = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_2'])
    # RI three year
    df_RI_three = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_3'])

    # Prediction datasets
    X_pred_HVZs_2019 = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'tic', 'cik', 'fyear','mrk_eq'])
    X_pred_HVZs_2022 = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'tic', 'cik', 'fyear','mrk_eq'])

    X_pred_EPs = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E'])
    X_pred_RIs = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC'])

    # Actual earnings datasets
    Y_pred_one = pd.DataFrame(columns=['tic','E_deflat'])
    Y_pred_two = pd.DataFrame(columns=['tic','E_deflat'])
    Y_pred_three = pd.DataFrame(columns=['tic','E_deflat'])
    count = 0
    
    for cik in company_list['cik'].astype(str):
        count += 1
        infocomp = inforaw.loc[(inforaw['cik'] == cik)].reset_index(drop=True)
        infocomp = infocomp.sort_values(by='fyear')

        # HVZ model
        infocomp['E'] = infocomp['ib'] - infocomp['spi']
        infocomp['DD'] = (infocomp['dvc'] > 0).astype(int)
        infocomp['NegE'] = (infocomp['E'] < 0).astype(int)
        infocomp['AC'] = (infocomp['act'] - infocomp['che']).diff() - (infocomp['lct'] - infocomp['dlc'] - infocomp['txp']).diff() - infocomp['dp']

        # EP model
        infocomp['E_deflat'] = infocomp['E'] / infocomp['csho']
        infocomp['NegE_E'] = infocomp['E_deflat'] * infocomp['NegE']

        # RI model
        infocomp['B'] = infocomp['ceq'] / infocomp['csho']
        infocomp['WC'] = (infocomp['act'] - infocomp['che']) - (infocomp['lct'] - infocomp['dlc'])
        infocomp['NCO'] = (infocomp['at'] - infocomp['act'] - infocomp['ivao']) - (infocomp['lt'] - infocomp['lct'] - infocomp['dltt'])
        infocomp['FIN'] = ((infocomp['ivst'] - infocomp['ivao']) - (infocomp['dltt'] - infocomp['dlc'] - infocomp['pstk'])) 
        infocomp['TACC'] = (infocomp['WC'] + infocomp['NCO'] + infocomp['FIN']) / infocomp['csho']

        # Additional Features
        # Calculate Shareholder's Equity
        infocomp['shareholders_equity'] = infocomp['at'] - infocomp['lt']

        # Calculate current ratio
        infocomp['current_ratio'] = infocomp['act'] / infocomp['lct']

        # Calculate debt to equity
        infocomp['debt_to_equity'] = infocomp['lt'] / infocomp['shareholders_equity']

        # Calculate return on equity (ROE)
        infocomp['roe'] = infocomp['ni'] / infocomp['shareholders_equity']
        
        # Calculate Asset Turnover
        infocomp['asset_turnover'] = infocomp['revt'] / infocomp['at']

        # Compile dataset
        # Drop year differnece axillary 
        infocomp = infocomp[infocomp['fyear'] != 2009]
        
        # Q5 2022 prediction
        X_pred_2022 = infocomp.loc[infocomp['fyear'] == 2022].reset_index(drop=True)
        X_pred_2022 = X_pred_2022.dropna()
        X_pred_HVZ_2022 = X_pred_2022[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'tic', 'cik', 'fyear','mrk_eq']].reset_index(drop=True)
        X_pred_HVZs_2022 = X_pred_HVZs_2022.append(X_pred_HVZ_2022, ignore_index=True)
        # Year range of training
        infocomptrain = infocomp[(infocomp['fyear'] >= 2010) & (infocomp['fyear'] <= 2019)].reset_index(drop=True)
        # Prediction data
        X_pred_2019 = infocomptrain.loc[infocomptrain['fyear'] == 2019].reset_index(drop=True)
        X_pred_HVZ_2019 = X_pred_2019[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'tic', 'cik', 'fyear','mrk_eq']].reset_index(drop=True)

        X_pred_HVZ_2019 = X_pred_HVZ_2019.dropna()
        X_pred_HVZs_2019 = X_pred_HVZs_2019.append(X_pred_HVZ_2019, ignore_index=True)
        if X_pred_2019.empty is not None: 
            # Yearly data
            Y_pred_2020 = infocomp[(infocomp['fyear'] == 2020)].reset_index(drop=True)
            Y_pred_2020 = Y_pred_2020[['tic','E_deflat']]
            Y_pred_one = Y_pred_one.append(Y_pred_2020, ignore_index=True)

            Y_pred_2021 = infocomp[(infocomp['fyear'] == 2021)].reset_index(drop=True)
            Y_pred_2021 = Y_pred_2021[['tic','E_deflat']]
            Y_pred_two = Y_pred_two.append(Y_pred_2021, ignore_index=True)

            Y_pred_2022 = infocomp[(infocomp['fyear'] == 2022)].reset_index(drop=True)
            Y_pred_2022 = Y_pred_2022[['tic','E_deflat']]
            Y_pred_three = Y_pred_three.append(Y_pred_2022, ignore_index=True)
            
        ## Calculate HVZ Prediction, One year gap
        # X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        # Since HVZ one is selected as the best model, it is fed with more variables for questions later
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'tic', 'cik', 'fyear','mrk_eq']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E']].reset_index(drop=True)

        y = y.rename(columns={"E": "E_1"})
        HVZ_one = pd.concat([X, y], axis=1)
        df_HVZ_one = df_HVZ_one.append(HVZ_one, ignore_index=True)
        df_HVZ_one = df_HVZ_one.dropna()

        ## Calculate HVZ Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E']].reset_index(drop=True)
        y = y.rename(columns={"E": "E_2"})
        HVZ_two = pd.concat([X, y], axis=1)
        df_HVZ_two = df_HVZ_two.append(HVZ_two, ignore_index=True)
        df_HVZ_two = df_HVZ_two.dropna()

        ## Calculate HVZ Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E']].reset_index(drop=True)
        y = y.rename(columns={"E": "E_3"})
        HVZ_three = pd.concat([X, y], axis=1)
        df_HVZ_three = df_HVZ_three.append(HVZ_three, ignore_index=True)
        df_HVZ_three = df_HVZ_three.dropna()

        ## Calculate EP Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E_deflat']].reset_index(drop=True)

        X_pred_EP = X_pred_2019[['E_deflat', 'NegE', 'NegE_E', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_EPs = X_pred_EPs.append(X_pred_EP, ignore_index=True)

        y = y.rename(columns={"E_deflat": "E_1"})
        EP_one = pd.concat([X, y], axis=1)
        df_EP_one = df_EP_one.append(EP_one, ignore_index=True)
        df_EP_one = df_EP_one.dropna()

        ## Calculate EP Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_2"})
        EP_two = pd.concat([X, y], axis=1)
        df_EP_two = df_EP_two.append(EP_two, ignore_index=True)
        df_EP_two = df_EP_two.dropna()

        ## Calculate EP Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_3"})
        EP_three = pd.concat([X, y], axis=1)
        df_EP_three = df_EP_three.append(EP_three, ignore_index=True)
        df_EP_three = df_EP_three.dropna()

        ## Calculate RI Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E_deflat']].reset_index(drop=True)
        
        X_pred_RI = X_pred_2019[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_RIs = X_pred_RIs.append(X_pred_RI, ignore_index=True)
        
        y = y.rename(columns={"E_deflat": "E_1"})
        RI_one = pd.concat([X, y], axis=1)
        df_RI_one = df_RI_one.append(RI_one, ignore_index=True)
        df_RI_one = df_RI_one.dropna()

        ## Calculate RI Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_2"})
        RI_two = pd.concat([X, y], axis=1)
        df_RI_two = df_RI_two.append(RI_two, ignore_index=True)
        df_RI_two = df_RI_two.dropna()

        ## Calculate RI Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_3"})
        RI_three = pd.concat([X, y], axis=1)
        df_RI_three = df_RI_three.append(RI_three, ignore_index=True)
        df_RI_three = df_RI_three.dropna()
    return df_HVZ_one, df_HVZ_two, df_HVZ_three, df_EP_one, df_EP_two, df_EP_three, df_RI_one, df_RI_two, df_RI_three, X_pred_HVZs_2019, X_pred_HVZs_2022, X_pred_EPs, X_pred_RIs, Y_pred_one, Y_pred_two, Y_pred_three


In [39]:
#Compile companylist for companies that EXISTED in EVERY year between startyeartrain to endyeartest to not Nan the value
year = startYear
query_comp = (
        f"""
        SELECT DISTINCT cik
        FROM comp.funda
        WHERE fyear = {year}
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        """) 
company_list = db.raw_sql(query_comp) 
for year in np.arange(startYear,endYear):
    query_comp = (
        f"""
        SELECT DISTINCT cik
        FROM comp.funda
        WHERE fyear = {year}
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        """) 
    comp_list = db.raw_sql(query_comp) 
    company_list = pd.merge(comp_list,company_list, on= 'cik', how='inner')
company_list = company_list.dropna().reset_index(drop=True)

# Company list obtained, Get company tic for reviewing convenience
query_comp = (
        f"""
        SELECT DISTINCT cik, MIN (tic) AS tic
        FROM comp.funda
        WHERE fyear BETWEEN '{startYear}' AND '{endYear}'
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        group by cik
        """) 
company_tic_list = db.raw_sql(query_comp) 
company_list = pd.merge(company_list, company_tic_list, left_on = 'cik', right_on = 'cik', how='left')
linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")# Make sure we have the tic exist in linkdf so we can find analyst data
company_list = pd.merge(company_list, linkdf, left_on = 'tic', right_on = 'TICKER', how='inner')

print(company_list)

             cik   tic   gvkey TICKER
0     0001119774  CGEN    3172   CGEN
1     0001115837   MBT  137433    MBT
2     0000075362  PCAR    8253   PCAR
3     0001395942   KAR  183581    KAR
4     0000871763   MAN    7007    MAN
...          ...   ...     ...    ...
1854  0001289848  HURN  161997   HURN
1855  0001084048    ZD   66716     ZD
1856  0001345105   CPA   21381    CPA
1857  0000882361  APTO   15490   APTO
1858  0001046025  HFWA   66285   HFWA

[1859 rows x 4 columns]


In [40]:
# Data extraction
df_HVZ_one, df_HVZ_two, df_HVZ_three, df_EP_one, df_EP_two, df_EP_three, df_RI_one, df_RI_two, df_RI_three, X_pred_HVZs_2019, X_pred_HVZs_2022, X_pred_EPs, X_pred_RIs, Y_pred_one, Y_pred_two, Y_pred_three = \
    calValues(db, company_list)

Estimated waiting time 4mins, please wait


In [41]:
# The result of this cell is saved to a csv and read instead of rerun it. 


# # How to use the object to generate sentiment analysis

# # This is to generate training set with  text analysis for year 2010-2018

# # Takes 10 hours

# # 1. Load the dictionary as dataframe
# lm_word_list_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
# # 2. Create a TextSentimentAnalyser object
# text_analyzer = TextSentimentAnalyser(lm_word_list_df)
# df_HVZ_one['sentiment'] = None
# for index,row in df_HVZ_one.iterrows():    
#     # 3. Pass in company ticker and year to calculate sentiment. If company does not have 10k
#     # that year it will return null
#     print(int(row['cik']))
#     sentiment = text_analyzer.get_sentiment(row['cik'], int(row['fyear']))
#     df_HVZ_one.at[index, 'sentiment'] = sentiment
#     print(sentiment)
# # Save the output as a csv to not rerun for 10 hours
# df_HVZ_one.to_csv('df_HVZ_one_additional_feature.csv', index=False)

In [42]:
# The result of this cell is saved to a csv and read instead of rerun it. 


# # This is to generate teesting set with text analysis for year 2019
# Only first 10% is tested

# # 1. Load the dictionary as dataframe
# lm_word_list_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
# # 2. Create a TextSentimentAnalyser object
# text_analyzer = TextSentimentAnalyser(lm_word_list_df)
# X_pred_HVZs_2019['sentiment'] = None
# for index,row in X_pred_HVZs_2019[:130].iterrows():    
#     # 3. Pass in company ticker and year to calculate sentiment. If company does not have 10k
#     # that year it will return null
#     print(int(row['cik']))
#     sentiment = text_analyzer.get_sentiment(row['cik'], int(row['fyear']))
#     X_pred_HVZs_2019.at[index, 'sentiment'] = sentiment
# # Save the output as a csv
# X_pred_HVZs_2019.to_csv('X_pred_HVZs_2019_additional_feature.csv', index=False)

In [43]:
X_pred_HVZs_2019[:130]

,at,dvc,DD,E,NegE,AC,csho,current_ratio,debt_to_equity,roe,asset_turnover,tic,cik,fyear,mrk_eq
0,53.763,0.000,0,-27.337,1,2.999,67.9230,6.599208,0.402964,-0.713369,0.000000,CGEN,0001119774,2019.0,2.300237e+08
1,13270.208,1196.736,1,855.683,0,-1646.385,886.4170,0.558428,21.638607,1.490382,0.577862,MBT,0001115837,2019.0,1.860494e+10
2,6581.200,142.300,1,139.900,0,-269.900,128.8330,1.288676,2.988123,0.114229,0.422704,KAR,0001395942,2019.0,3.335750e+09
3,9223.800,129.300,1,460.900,0,-156.200,58.6740,1.460013,2.340141,0.168640,2.261920,MAN,0000871763,2019.0,5.778709e+09
4,1499.700,0.000,0,-86.700,1,-112.900,89.1860,2.085152,1.221778,-0.119704,1.224645,CENX,0000949157,2019.0,6.137255e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,1692.382,0.000,0,156.246,0,-22.036,20.7420,3.550546,0.228818,0.073360,0.748181,ICUI,0000883984,2019.0,5.192873e+09
126,3406.759,8.544,1,136.788,0,25.067,45.4580,1.906810,2.527327,-0.390358,0.625603,BDC,0000913142,2019.0,2.350334e+09
127,909.113,0.000,0,70.758,0,45.073,27.0000,1.680632,0.870045,0.142102,1.747202,PLUS,0001022408,2019.0,9.382734e+08
128,1744.376,153.836,1,255.949,0,-68.525,491.1465,0.755514,1.138371,0.249276,1.155414,ROL,0000084839,2019.0,1.174850e+10


## Build Model

In [44]:
# Prepare dataset for nine models
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one, 'HVZ_two': df_HVZ_two, 'HVZ_three': df_HVZ_three,
        'EP_one': df_EP_one, 'EP_two': df_EP_two, 'EP_three': df_EP_three,
        'RI_one': df_RI_one, 'RI_two': df_RI_two, 'RI_three': df_RI_three}

for name, df in data.items(): 
    print(name)
    if 'HVZ' in name:
        X = df[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
    elif 'EP' in name:
        X = df[['E_deflat', 'NegE', 'NegE_E']]
    elif 'RI' in name:
        X = df[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']]
    
    if 'one' in name:
        y = df['E_1']
    elif 'two' in name:
        y = df['E_2']
    elif 'three' in name:
        y = df['E_3']

    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    if 'HVZ' in name:
        y_pred_adjusted = y_pred / df['csho']
        y_pred_adjusted_dict[name] = y_pred_adjusted


HVZ_one
Adjusted R-squared Score:  0.8406212032241767
HVZ_two
Adjusted R-squared Score:  0.7651757641499377
HVZ_three
Adjusted R-squared Score:  0.7206803654926939
EP_one
Adjusted R-squared Score:  0.6517038844402605
EP_two
Adjusted R-squared Score:  0.37627104332115535
EP_three
Adjusted R-squared Score:  0.09523489577762567
RI_one
Adjusted R-squared Score:  0.6586419773745988
RI_two
Adjusted R-squared Score:  0.527852990597225
RI_three
Adjusted R-squared Score:  0.14284848752340373


In [45]:
# 2020, 2021, 2023 Prediction using all models and 2019 data
models = {'HVZ_one': models['HVZ_one'], 'EP_one': models['EP_one'], 'RI_one': models['RI_one'],
          'HVZ_two': models['HVZ_two'], 'EP_two': models['EP_two'], 'RI_two': models['RI_two'],
          'HVZ_three': models['HVZ_three'], 'EP_three': models['EP_three'], 'RI_three': models['RI_three']}


X_2019 = df_HVZ_one[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]

for model_name, model in models.items():
    print(model_name)
    if 'HVZ' in model_name:
        X = X_pred_HVZs_2019[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
    elif 'EP' in model_name:
        X = X_pred_EPs[['E_deflat', 'NegE', 'NegE_E']]
    elif 'RI' in model_name:
        X = X_pred_RIs[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']]
    
    X = add_constant(X)
    
    if 'one' in model_name:
        y = Y_pred_one.copy()
    elif 'two' in model_name:
        y = Y_pred_two.copy()
    elif 'three' in model_name:
        y = Y_pred_three.copy()
        
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred_EPS'])
    y_pred_df['y_pred_EPS'] = y_pred_df['y_pred_EPS'] / X_pred_HVZs_2019['csho']
 
    y_pred_df = pd.concat([X_pred_HVZs_2019['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred_EPS'])/(X_pred_HVZs_2019['mrk_eq']/X_pred_HVZs_2019['csho']/1000000)
    y_pred_df = y_pred_df.rename(columns={"E_deflat": "Actual_EPS_for_the_predicted_year"})

    y_pred_df['accuracy'] = np.absolute(y_pred_df['bias'])
    y_pred_df = y_pred_df.sort_values(by='accuracy', ascending=True)
    # print(y_pred_df.head(10))
    print('\nOverrall Mean Bias:')
    print(y_pred_df['bias'].mean())
    print('\nOverrall Median Bias:')
    print(y_pred_df['bias'].median())
    print('\nOverrall Mean Accuracy:')
    print(y_pred_df['accuracy'].mean())
    print('\nOverrall Median Accuracy:')
    print(y_pred_df['accuracy'].median())

HVZ_one

Overrall Mean Bias:
-3.1156182472830305

Overrall Median Bias:
-0.02081008183408248

Overrall Mean Accuracy:
3.6011608848489995

Overrall Median Accuracy:
0.03902814637736633
EP_one

Overrall Mean Bias:
12.886871536038438

Overrall Median Bias:
0.041126799676829855

Overrall Mean Accuracy:
13.06593149174973

Overrall Median Accuracy:
0.0630559403448498
RI_one

Overrall Mean Bias:
12.947366494849916

Overrall Median Bias:
0.04136821204775006

Overrall Mean Accuracy:
13.120544419947004

Overrall Median Accuracy:
0.06372286258976068
HVZ_two

Overrall Mean Bias:
-14.266930297889946

Overrall Median Bias:
-0.01373952514803531

Overrall Mean Accuracy:
14.669871631001108

Overrall Median Accuracy:
0.06163219701998613
EP_two

Overrall Mean Bias:
-41.35977527703941

Overrall Median Bias:
0.06430552142676854

Overrall Mean Accuracy:
44.19522864562778

Overrall Median Accuracy:
0.08870902071404145
RI_two

Overrall Mean Bias:
-41.40138423918443

Overrall Median Bias:
0.06286236432815047



## Q2

### Without sentiment

In [46]:
#loading saved training set, which contains sentiment column
df_HVZ_one_additional_feature = pd.read_csv('df_HVZ_one_additional_feature.csv')
# print(df_HVZ_one_additional_feature)
# get dummy variables for the 'color' column
df_sentiment_dummy = pd.get_dummies(df_HVZ_one_additional_feature['sentiment'], prefix='sentiment')

# concatenate the dummy variables with the original DataFrame
df_HVZ_one_additional_feature = pd.concat([df_HVZ_one_additional_feature, df_sentiment_dummy], axis=1)
df_HVZ_one_additional_feature
df_HVZ_one_additional_feature_train = df_HVZ_one_additional_feature.drop('sentiment', axis=1)
# print(df_HVZ_one_additional_feature_train)

X_pred_HVZs_2019 = pd.read_csv('X_pred_HVZs_2019_additional_feature.csv')

X_pred_HVZs_2019_sentiment = X_pred_HVZs_2019[:100]
# get dummy variables for the 'color' column
df_sentiment_dummy = pd.get_dummies(X_pred_HVZs_2019_sentiment['sentiment'], prefix='sentiment')

# concatenate the dummy variables with the original DataFrame
X_pred_HVZs_2019_sentiment = pd.concat([X_pred_HVZs_2019_sentiment, df_sentiment_dummy], axis=1)
X_pred_HVZs_2019_sentiment
X_pred_HVZs_2019_sentiment_test = df_HVZ_one_additional_feature.drop('sentiment', axis=1)
X_pred_HVZs_2019_sentiment_test

,at,dvc,DD,E,NegE,AC,current_ratio,debt_to_equity,roe,asset_turnover,tic,cik,fyear,mrk_eq,E_1,csho,sentiment_Litigious,sentiment_Negative,sentiment_NoSentiment,sentiment_Positive,sentiment_The 10k is in txt instead of html,sentiment_Uncertainty
0,36.458,0.0,0,-7.203,1,-4.113,11.502253,0.288952,-0.254658,0.030583,CGEN,1119774,2010,110286270.0,-10.704,33.916,0,0,1,0,0,0
1,29.081,0.0,0,-10.704,1,-4.468,13.479203,0.485164,-0.613043,0.000000,CGEN,1119774,2011,138673360.0,-13.628,34.708,0,0,1,0,0,0
2,28.909,0.0,0,-13.628,1,0.168,18.476156,0.635865,-0.771163,0.008371,CGEN,1119774,2012,138732260.0,-14.083,36.590,0,0,1,0,0,0
3,56.711,0.0,0,-14.083,1,-5.684,6.876341,0.778443,-0.441639,0.062580,CGEN,1119774,2013,206220540.0,-10.302,41.002,0,0,1,0,0,0
4,114.986,0.0,0,-10.302,1,-0.381,11.397481,0.083588,-0.104546,0.107552,CGEN,1119774,2014,435682370.0,-20.163,50.254,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12148,31.600,0.0,0,-11.992,1,0.344,13.487973,0.079530,-0.409675,0.000000,APTO,882361,2014,57965505.0,-14.626,11.700,0,0,1,0,0,0
12149,21.249,0.0,0,-14.626,1,-0.322,8.834890,0.124702,-0.774149,0.000000,APTO,882361,2015,60751200.0,-18.627,12.048,0,0,1,0,0,0
12150,11.610,0.0,0,-18.627,1,0.028,6.398305,0.179878,-1.892988,0.000000,APTO,882361,2016,29823850.0,-11.661,15.721,0,0,1,0,0,0
12151,11.967,0.0,0,-11.661,1,-0.348,6.699717,0.173005,-1.143011,0.000000,APTO,882361,2017,26757420.0,-23.868,27.502,0,0,1,0,0,0


In [47]:
# Prepare dataset for nine models
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one_additional_feature_train}

for name, df in data.items(): 
    print(name)
    X = df[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover']]
    y = df['E_1']

    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    # print('Bias:')
    # print(y - y_pred) 
    # Calculate R-squared
    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    # coefficients = pd.DataFrame({"Variable": X.columns, "Coefficient": model.coef_})
    # print(coefficients)

    if 'HVZ' in name:
        y_pred_adjusted = y_pred / df['csho']
        y_pred_adjusted_dict[name] = y_pred_adjusted

HVZ_one
Adjusted R-squared Score:  0.8406057040979039


In [48]:
# 2020, 2021, 2023 Prediction using all models and 2019 data
models = {'HVZ_one': models['HVZ_one']}

for model_name, model in models.items():
    print(model_name)
    X = X_pred_HVZs_2019[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover']]
    
    X = add_constant(X)
    
    y = Y_pred_one.copy()
        
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred_EPS'])
    y_pred_df['y_pred_EPS'] = y_pred_df['y_pred_EPS'] / X_pred_HVZs_2019['csho']
 
    y_pred_df = pd.concat([X_pred_HVZs_2019['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred_EPS'])/(X_pred_HVZs_2019['mrk_eq']/X_pred_HVZs_2019['csho']/1000000)
    y_pred_df = y_pred_df.rename(columns={"E_deflat": "Actual_EPS_for_the_predicted_year"})

    y_pred_df['accuracy'] = np.absolute(y_pred_df['bias'])
    y_pred_df = y_pred_df.sort_values(by='accuracy', ascending=True)
    # print(y_pred_df.head(10))
    print('\nOverrall Mean Bias:')
    print(y_pred_df['bias'].mean())
    print('\nOverrall Median Bias:')
    print(y_pred_df['bias'].median())
    print('\nOverrall Mean Accuracy:')
    print(y_pred_df['accuracy'].mean())
    print('\nOverrall Median Accuracy:')
    print(y_pred_df['accuracy'].median())

HVZ_one

Overrall Mean Bias:
-3.053697367127171

Overrall Median Bias:
-0.020344921678304692

Overrall Mean Accuracy:
3.5370993807906084

Overrall Median Accuracy:
0.0389887715906908


### Without sentiment, first 130 companies 

In [49]:
# To compare the performance of model with or without sentiment analysis
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one_additional_feature_train[:130]}

for name, df in data.items(): 
    print(name)
    X = df[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover']]
    y = df['E_1']

    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    if 'HVZ' in name:
        y_pred_adjusted = y_pred / df['csho']
        y_pred_adjusted_dict[name] = y_pred_adjusted

HVZ_one
Adjusted R-squared Score:  0.8659561131131688


In [50]:
# 2020 Prediction using all models and 2019 data
models = {'HVZ_one': models['HVZ_one']}

for model_name, model in models.items():
    print(model_name)

    X = X_pred_HVZs_2019[:130][['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover']]
    
    X = add_constant(X)
    
    y = Y_pred_one.copy()
        
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred_EPS'])
    y_pred_df['y_pred_EPS'] = y_pred_df['y_pred_EPS'] / X_pred_HVZs_2019['csho']
 
    y_pred_df = pd.concat([X_pred_HVZs_2019['tic'], y_pred_df], axis=1)
    y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred_EPS'])/(X_pred_HVZs_2019['mrk_eq']/X_pred_HVZs_2019['csho']/1000000)
    y_pred_df = y_pred_df.rename(columns={"E_deflat": "Actual_EPS_for_the_predicted_year"})

    y_pred_df['accuracy'] = np.absolute(y_pred_df['bias'])
    y_pred_df = y_pred_df.sort_values(by='accuracy', ascending=True)
    # print(y_pred_df.head(10))
    print('\nOverrall Mean Bias:')
    print(y_pred_df['bias'].mean())
    print('\nOverrall Median Bias:')
    print(y_pred_df['bias'].median())
    print('\nOverrall Mean Accuracy:')
    print(y_pred_df['accuracy'].mean())
    print('\nOverrall Median Accuracy:')
    print(y_pred_df['accuracy'].median())

HVZ_one

Overrall Mean Bias:
-0.35712898604497606

Overrall Median Bias:
-0.03691403719790487

Overrall Mean Accuracy:
0.6246216551888827

Overrall Median Accuracy:
0.06440779412830523


### With sentiment

In [51]:
# Prepare dataset for nine models
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one_additional_feature_train[:130]}

for name, df in data.items(): 
    print(name)
    X = df[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'sentiment_Litigious', 'sentiment_Negative', 'sentiment_NoSentiment', 'sentiment_Positive', 'sentiment_Uncertainty' ]]
    y = df['E_1']

    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    if 'HVZ' in name:
        y_pred_adjusted = y_pred / df['csho']
        y_pred_adjusted_dict[name] = y_pred_adjusted


HVZ_one
Adjusted R-squared Score:  0.8614939017050207


In [52]:
# 2020 Prediction using all models and 2019 data
models = {'HVZ_one': models['HVZ_one']}

for model_name, model in models.items():
    print(model_name)
    X = X_pred_HVZs_2019_sentiment_test[:130][['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'sentiment_Litigious', 'sentiment_Negative', 'sentiment_NoSentiment', 'sentiment_Positive', 'sentiment_Uncertainty']]
    
    X = add_constant(X)
    
    y = Y_pred_one.copy()
 
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred_EPS'])
    y_pred_df['y_pred_EPS'] = y_pred_df['y_pred_EPS'] / X_pred_HVZs_2019['csho']
 
    y_pred_df = pd.concat([X_pred_HVZs_2019['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred_EPS'])/(X_pred_HVZs_2019['mrk_eq']/X_pred_HVZs_2019['csho']/1000000)
    y_pred_df = y_pred_df.rename(columns={"E_deflat": "Actual_EPS_for_the_predicted_year"})

    y_pred_df['accuracy'] = np.absolute(y_pred_df['bias'])
    y_pred_df = y_pred_df.sort_values(by='accuracy', ascending=True)
    # print(y_pred_df.head(10))
    print('\nOverrall Mean Bias:')
    print(y_pred_df['bias'].mean())
    print('\nOverrall Median Bias:')
    print(y_pred_df['bias'].median())
    print('\nOverrall Mean Accuracy:')
    print(y_pred_df['accuracy'].mean())
    print('\nOverrall Median Accuracy:')
    print(y_pred_df['accuracy'].median())

HVZ_one

Overrall Mean Bias:
-5.056318499585114

Overrall Median Bias:
-0.09324247364028544

Overrall Mean Accuracy:
5.151537200231966

Overrall Median Accuracy:
0.11539534104939836


## Q3

In [53]:
# Prepare dataset for nine models
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one}
for name, df in data.items(): 
    print(name)
    X = X_2019
    y = df['E_1']
   
    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    # print('Bias:')
    # print(y - y_pred) 
    # Calculate R-squared
    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    # coefficients = pd.DataFrame({"Variable": X.columns, "Coefficient": model.coef_})
    # print(coefficients)

    if 'HVZ' in name:
        y_pred_adjusted = y_pred / df['csho']
        y_pred_adjusted_dict[name] = y_pred_adjusted

HVZ_one
Adjusted R-squared Score:  0.8406212032241767


In [54]:
# 2020 Prediction using 2019 data for Q3

models = {'HVZ_one': models['HVZ_one']}

for model_name, model in models.items():
    print(model_name)
    X = X_pred_HVZs_2019[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
 
    X = add_constant(X)
    y = Y_pred_one.copy()
   
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred_EPS'])
    y_pred_df['y_pred_EPS'] = y_pred_df['y_pred_EPS'] / X_pred_HVZs_2019['csho']
 
    y_pred_df = pd.concat([X_pred_HVZs_2019['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred_EPS'])/(X_pred_HVZs_2019['mrk_eq']/X_pred_HVZs_2019['csho']/1000000)
    y_pred_df = y_pred_df.rename(columns={"E_deflat": "Actual_EPS_for_the_predicted_year"})

    y_pred_df['accuracy'] = np.absolute(y_pred_df['bias'])
    y_pred_df = y_pred_df.sort_values(by='accuracy', ascending=True)
    print(y_pred_df.head(20))
    print('\nOverrall Mean Bias:')
    print(y_pred_df['bias'].mean())
    print('\nOverrall Median Bias:')
    print(y_pred_df['bias'].median())
    print('\nOverrall Mean Accuracy:')
    print(y_pred_df['accuracy'].mean())
    print('\nOverrall Median Accuracy:')
    print(y_pred_df['accuracy'].median())

HVZ_one
       tic  y_pred_EPS  Actual_EPS_for_the_predicted_year      bias  accuracy
414    UNP    8.291144                           8.292235  0.000006  0.000006
1355  JBHT    4.983436                           4.983578  0.000007  0.000007
652    KSU    7.110750                           7.112810  0.000016  0.000016
1351  TTMI    0.718713                           0.714892 -0.000037  0.000037
1156  OTEX    1.298169                           1.299608  0.000050  0.000050
1308   CNX    0.064587                          -0.043368 -0.000072  0.000072
758   ASGN    3.911728                           3.916824  0.000084  0.000084
937   GRMN    5.059278                           5.179928  0.000095  0.000095
700     BR    4.282887                           4.296264  0.000103  0.000103
827   IRWD    0.744771                           0.756819  0.000103  0.000103
939    IPG    1.970870                           1.956766 -0.000154  0.000154
30    SMCI    2.552189                           2.55538

In [55]:
prediction_2019 = pd.merge(y_pred_df, X_pred_HVZs_2019, how='inner', left_on=['tic'], right_on=['tic'])
prediction_2019 = prediction_2019.sort_values(by='accuracy', ascending=True)

prediction_2019.head(30)
# Table of 2019 prediction with analyst data 

,tic,y_pred_EPS,Actual_EPS_for_the_predicted_year,bias,accuracy,at,dvc,DD,E,NegE,AC,csho,current_ratio,debt_to_equity,roe,asset_turnover,cik,fyear,mrk_eq,sentiment
0,UNP,8.291144,8.292235,0.000006,0.000006,61673.000,2598.000,1,5835.000,0,-2219.000,692.101,0.794990,2.402085,0.326511,0.351985,100885,2019.0,1.197025e+11,NaN
1,JBHT,4.983436,4.983578,0.000007,0.000007,5470.854,111.817,1,583.920,0,-437.951,106.213,1.430148,1.413226,0.227752,1.675288,728535,2019.0,9.939832e+09,NaN
2,KSU,7.110750,7.112810,0.000016,0.000016,9786.800,146.500,1,703.800,0,-297.600,96.116,1.483004,1.062159,0.113551,0.292843,54480,2019.0,1.225351e+10,NaN
3,TTMI,0.718713,0.714892,-0.000037,0.000037,3560.933,0.000,0,55.184,0,-271.639,105.510,1.418329,1.784073,0.032291,0.755226,1116942,2019.0,1.075702e+09,NaN
4,OTEX,1.298169,1.299608,0.000050,0.000050,7933.975,168.859,1,321.220,0,-520.441,269.032,1.538683,1.042381,0.073494,0.361579,1002638,2019.0,1.110282e+10,NaN
5,CNX,0.064587,-0.043368,-0.000072,0.000072,9060.806,0.000,0,-75.275,1,-574.689,186.643,0.941698,0.825925,-0.016269,0.212381,1070412,2019.0,1.428871e+09,NaN
6,ASGN,3.911728,3.916824,0.000084,0.000084,2941.400,0.000,0,203.600,0,-48.508,52.900,2.321795,1.137335,0.126944,1.334025,890564,2019.0,3.199680e+09,NaN
7,GRMN,5.059278,5.179928,0.000095,0.000095,6166.799,434.044,1,952.486,0,180.440,190.686,2.951784,0.286493,0.198704,0.609312,1121788,2019.0,1.514979e+10,NaN
8,BR,4.282887,4.296264,0.000103,0.000103,3880.700,224.200,1,482.100,0,-112.600,114.300,1.298654,2.441863,0.427583,1.124076,1383312,2019.0,1.482863e+10,NaN
9,IRWD,0.744771,0.756819,0.000103,0.000103,402.748,0.000,0,108.325,0,63.100,157.536,7.497178,-5.318967,-0.230614,1.063725,1446847,2019.0,1.702494e+09,NaN


## Q4

In [57]:
X_pred_HVZs_2022_select = X_pred_HVZs_2022

In [58]:
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one_additional_feature_train}

for name, df in data.items(): 
    print(name)
    X = df_HVZ_one_additional_feature_train[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover']]
    y = df['E_1']
   
    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    # print('Bias:')
    # print(y - y_pred) 
    # Calculate R-squared
    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    # coefficients = pd.DataFrame({"Variable": X.columns, "Coefficient": model.coef_})
    # print(coefficients)

    if 'HVZ' in name:
        y_pred_adjusted = y_pred / df['csho']
        y_pred_adjusted_dict[name] = y_pred_adjusted

HVZ_one
Adjusted R-squared Score:  0.8406057040979039


In [59]:
# 2022 Prediciton, Total earning prediction
models = {'HVZ_one': models['HVZ_one']}

for model_name, model in models.items():
    print(model_name)
    X = X_pred_HVZs_2022_select[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover']]
 
    X = add_constant(X)
   
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred_EPS'])
    y_pred_df['y_pred_EPS'] = y_pred_df['y_pred_EPS'] / X_pred_HVZs_2022_select['csho']
 
    y_pred_df = pd.concat([X_pred_HVZs_2022['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    # y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    # y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred'])/(X_pred_HVZs_2022_select['mrk_eq']/X_pred_HVZs_2022_select['csho']/1000000)
    # y_pred_df['accuracy'] = np.absolute(y_pred_df['bias'])
    y_pred_df['Total_Earning_Growth'] = y_pred_df['y_pred_EPS'] * X_pred_HVZs_2022_select['csho'] - X['E']
    y_pred_df = y_pred_df.sort_values(by='Total_Earning_Growth', ascending=False)
    print(y_pred_df.head(20))

HVZ_one
       tic  y_pred_EPS  Total_Earning_Growth
279   AMZN    0.100692           2653.284722
148    CME   12.081347           1639.181978
527    ICE    5.614372           1385.433870
585     BA   -3.015553           1379.935544
290    CCL   -3.457697           1298.759680
33     DUK    6.995144           1203.260930
7        D    2.680553           1128.261917
327    BTI    4.936890           1002.303833
558      F    2.386972            905.905083
743    BIO  -92.556194            873.167889
1069   BIP    2.776200            865.554546
620     SO    4.407696            830.980796
984    RTX    4.175438            734.194028
1052   IBM    9.016591            732.860635
732    BHP    8.670194            664.104865
1068   AMT    6.406401            638.741798
753     ED    6.411603            616.119030
619     PM    6.563938            614.535395
54     KMI    1.393927            585.104421
93     BEP    1.680737            564.806132


In [60]:
prediction_2022 = pd.merge(y_pred_df.head(20), X_pred_HVZs_2022_select, how='inner', left_on=['tic'], right_on=['tic'])
prediction_2022 = prediction_2022.sort_values(by='Total_Earning_Growth', ascending=False)
prediction_2022.head(20)

,tic,y_pred_EPS,Total_Earning_Growth,at,dvc,DD,E,NegE,AC,csho,current_ratio,debt_to_equity,roe,asset_turnover,cik,fyear,mrk_eq
0,AMZN,0.100692,2653.284722,462675.000,0.000,0,-1622.000,1,-21579.000,10242.000,0.944644,2.168074,-0.018638,1.110894,0001018724,2022.0,1.080624e+12
1,CME,12.081347,1639.181978,174175.700,3096.100,1,2697.200,0,-932.000,358.932,1.010101,5.480064,0.100116,0.028818,0001156375,2022.0,7.357286e+10
2,ICE,5.614372,1385.433870,194338.000,853.000,1,1753.000,0,-1273.000,559.000,1.052128,7.538201,0.063530,0.049584,0001571949,2022.0,5.249933e+10
3,BA,-3.015553,1379.935544,137100.000,0.000,0,-3182.000,1,-5919.000,597.590,1.216220,-9.650934,0.311396,0.485835,0000012927,2022.0,8.088847e+10
4,CCL,-3.457697,1298.759680,51703.000,0.000,0,-5652.000,1,-4103.000,1259.000,0.706459,6.318188,-0.862420,0.235344,0000815097,2022.0,8.560914e+09
5,DUK,6.995144,1203.260930,178086.000,3073.000,1,4183.000,0,-3435.000,770.000,0.700578,2.434440,0.049177,0.161798,0001326160,2022.0,8.254087e+10
6,D,2.680553,1128.261917,104243.000,2209.000,1,1110.000,0,-1355.000,835.000,0.732342,2.738854,0.035652,0.164750,0000715957,2022.0,6.629418e+10
7,BTI,4.936890,1002.303833,185007.575,5922.084,1,10006.694,0,-1134.391,2229.946,0.863104,1.028081,0.088046,0.180109,0001303523,2022.0,9.688379e+10
8,F,2.386972,905.905083,255884.000,2034.000,1,8609.000,0,-247.000,3986.182,1.202445,4.927769,-0.045892,0.617690,0000037996,2022.0,4.395140e+10
9,BIO,-92.556194,873.167889,13501.666,0.000,0,-3612.461,1,224.556,29.596,5.552895,0.404193,-0.377269,0.207548,0000012208,2022.0,1.230718e+10


In [61]:
Accuracy_2019 = prediction_2019[['tic', 'accuracy']]
prediction_2022 = pd.merge(prediction_2022.head(20), Accuracy_2019, how='inner', left_on=['tic'], right_on=['tic'])

prediction_2022.sort_values(by="Total_Earning_Growth", ascending=True, inplace=True)
prediction_2022.head(20)
# With accuracy from 2019 prediction

,tic,y_pred_EPS,Total_Earning_Growth,at,dvc,DD,E,NegE,AC,csho,current_ratio,debt_to_equity,roe,asset_turnover,cik,fyear,mrk_eq,accuracy
19,BEP,1.680737,564.806132,64111.000,355.000,1,-102.000,1,-1570.000,275.359,0.846247,1.438979,-0.004641,0.073482,0001533232,2022.0,9.573480e+09,0.037857
18,KMI,1.393927,585.104421,70078.000,2504.000,1,2548.000,0,-2233.000,2247.682,0.548773,1.182164,0.079342,0.273980,0001506307,2022.0,3.800285e+10,0.001703
17,PM,6.563938,614.535395,61681.000,7841.000,1,9561.000,0,-747.000,1550.218,0.717698,-10.773570,-1.433687,0.514940,0001413329,2022.0,1.530579e+11,0.005390
16,ED,6.411603,616.119030,69065.000,1120.000,1,1660.000,0,1069.000,355.000,1.144319,2.306286,0.079468,0.226888,0001047862,2022.0,3.369345e+10,0.010848
15,AMT,6.406401,638.741798,67194.500,2725.300,1,2344.200,0,-2537.400,465.619,0.435797,4.415199,0.142306,0.159404,0001053507,2022.0,1.166377e+11,1.780589
14,BHP,8.670194,664.104865,95166.000,17720.000,1,21277.000,0,-7626.000,2530.636,1.694190,0.951483,0.633638,0.698737,0000811809,2022.0,2.844013e+11,0.022806
13,IBM,9.016591,732.860635,127243.000,5948.000,1,7437.000,0,-5808.000,906.092,0.924234,4.778257,0.074429,0.475704,0000051143,2022.0,1.269912e+11,0.057963
12,RTX,4.175438,734.194028,158864.000,3128.000,1,5388.000,0,-4761.000,1466.240,1.085110,1.140620,0.070027,0.422210,0000101829,2022.0,1.429362e+11,0.146239
11,SO,4.407696,830.980796,134891.000,2907.000,1,3969.000,0,-3757.000,1089.000,0.662427,2.906261,0.102369,0.217057,0000092122,2022.0,7.576866e+10,0.003946
10,BIP,2.776200,865.554546,72969.000,1352.000,1,407.000,0,-263.000,458.380,0.798138,1.853249,0.015915,0.197714,0001406234,2022.0,1.750644e+10,0.424663


In [62]:
# 2022 Prediciton, EPS prediction
models = {'HVZ_one': models['HVZ_one']}

for model_name, model in models.items():
    print(model_name)
    X = X_pred_HVZs_2022_select[['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover']]
 
    X = add_constant(X)
   
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred_EPS'])
    y_pred_df['y_pred_EPS'] = y_pred_df['y_pred_EPS'] / X_pred_HVZs_2022_select['csho']
 
    y_pred_df = pd.concat([X_pred_HVZs_2022['tic'], y_pred_df], axis=1)

    y_pred_df['EPS_Prediction_Growth'] = y_pred_df['y_pred_EPS'] - X['E'] / X_pred_HVZs_2022_select['csho']
    y_pred_df = y_pred_df.sort_values(by='EPS_Prediction_Growth', ascending=False)
    print(y_pred_df.head(20))

HVZ_one
       tic  y_pred_EPS  EPS_Prediction_Growth
599   AMEN  308.593019             243.630755
271   FSCR   24.875666              30.469733
1123  MSTR  -96.961593              30.304577
743    BIO  -92.556194              29.502902
625   WEBC   92.225734              29.350572
482    CVR   25.638438              27.574256
684   TCCO   21.182561              22.718160
1164  MTEX   20.049996              22.465273
1065  ISIG   20.862514              21.949882
821    NBY   17.942814              19.812593
746   PSTV   11.321945              19.746770
673   REED   10.243509              17.871273
1169  DGLY    8.114083              16.803677
89    PFIN   12.088075              12.541112
137   DYNT   10.028939              11.440083
171    ADD   -5.113308              10.925682
531   AHPI    8.828392              10.163968
862   NSYS    9.453079               8.706145
75    LEDS    8.090083               8.657964
121    GHC   48.188333               8.632870


In [63]:
prediction_2022 = pd.merge(y_pred_df, df_analyst_raw_test_2022, how='inner', left_on=['tic'], right_on=['ticker'])
prediction_2022 = prediction_2022.sort_values(by='EPS_Prediction_Growth', ascending=False)
prediction_2022.head(20)

,tic,y_pred_EPS,EPS_Prediction_Growth,index,ticker,fpedats,statpers,actual,medest,fpi,gvkey,TICKER
0,BIO,-92.556194,29.502902,235,BIO,2022-12-31,2022-06-16,14.42,14.50,1,2220,BIO
1,DYNT,10.028939,11.440083,1144,DYNT,2022-06-30,2022-06-16,-1.30,-1.20,1,4124,DYNT
2,WHR,-6.910222,7.867556,2594,WHR,2022-12-31,2022-06-16,19.64,24.86,1,11465,WHR
3,PDEX,7.307377,6.235353,2461,PDEX,2022-06-30,2022-06-16,1.02,0.83,1,12458,PDEX
4,STRT,8.001564,6.187790,3325,STRT,2022-06-30,2022-06-16,1.80,2.51,1,31567,STRT
5,ACU,6.454200,5.596371,517,ACU,2022-12-31,2022-06-16,0.82,2.38,1,1104,ACU
6,FSTR,1.752897,4.583740,1571,FSTR,2022-12-31,2022-06-16,-4.25,0.31,1,4860,FSTR
7,CME,12.081347,4.566832,610,CME,2022-12-31,2022-06-16,7.98,7.83,1,149070,CME
8,USAP,4.163137,4.462916,3355,USAP,2022-12-31,2022-06-16,-0.90,-0.16,1,31170,USAP
9,FLXS,4.602481,4.379273,1374,FLXS,2022-06-30,2022-06-16,0.64,0.63,1,4771,FLXS


In [64]:
Accuracy_2019 = prediction_2019[['tic', 'accuracy']]
prediction_2022 = pd.merge(prediction_2022, Accuracy_2019, how='inner', left_on=['tic'], right_on=['tic'])
prediction_2022.head(20)

,tic,y_pred_EPS,EPS_Prediction_Growth,index,ticker,fpedats,statpers,actual,medest,fpi,gvkey,TICKER,accuracy
0,BIO,-92.556194,29.502902,235,BIO,2022-12-31,2022-06-16,14.42,14.50,1,2220,BIO,126.550573
1,DYNT,10.028939,11.440083,1144,DYNT,2022-06-30,2022-06-16,-1.30,-1.20,1,4124,DYNT,2.873078
2,WHR,-6.910222,7.867556,2594,WHR,2022-12-31,2022-06-16,19.64,24.86,1,11465,WHR,0.662729
3,PDEX,7.307377,6.235353,2461,PDEX,2022-06-30,2022-06-16,1.02,0.83,1,12458,PDEX,0.360203
4,STRT,8.001564,6.187790,3325,STRT,2022-06-30,2022-06-16,1.80,2.51,1,31567,STRT,0.446998
5,ACU,6.454200,5.596371,517,ACU,2022-12-31,2022-06-16,0.82,2.38,1,1104,ACU,0.219171
6,FSTR,1.752897,4.583740,1571,FSTR,2022-12-31,2022-06-16,-4.25,0.31,1,4860,FSTR,3.184023
7,CME,12.081347,4.566832,610,CME,2022-12-31,2022-06-16,7.98,7.83,1,149070,CME,0.010078
8,USAP,4.163137,4.462916,3355,USAP,2022-12-31,2022-06-16,-0.90,-0.16,1,31170,USAP,0.286482
9,FLXS,4.602481,4.379273,1374,FLXS,2022-06-30,2022-06-16,0.64,0.63,1,4771,FLXS,0.110471
